In [1]:
import numpy as np
from PIL import Image
import cv2
from matplotlib import pyplot as plt
import serial
import time

In [2]:

# /*
# RGBMODE
# 0 - 16bit (5-6-5)
# 1 - 8bit
# 2 - 24bit (8-8-8)
# */


def EightBitsTo5Bits(adc):
    return bin(int((adc * 31.0) / 255.0))[2:].zfill(5)


def EightBitsTo6Bits(adc):
    return bin(int((adc * 63.0) / 255.0))[2:].zfill(6)


def SixteenbitRGBtoSeperateRGB(a):
    rgb = bin(int(a, 16))[2:].zfill(16)
    r, g, b = int(rgb[0:4], 2)*255.0/31.0, int(rgb[5:10], 2) * \
        255.0/63.0, int(rgb[11:15], 2)*255.0/31.0
    return np.asarray([r, g, b])

In [3]:
def imgToMemoryFile(fileName='m2.jpg', WH={'W': 240, 'H': 120}, RGBMode=0):

    img = plt.imread(fileName)
    img = cv2.resize(img, (WH['W'], WH['H']))
    # plt.imshow(img)
    # plt.show()

    if RGBMode == 0:

        EightBitsTo5Bits_V = np.vectorize(EightBitsTo5Bits)
        EightBitsTo6Bits_V = np.vectorize(EightBitsTo6Bits)

        r = EightBitsTo5Bits_V(img[:, :, 0]).reshape(-1, 1)
        g = EightBitsTo6Bits_V(img[:, :, 1]).reshape(-1, 1)
        b = EightBitsTo5Bits_V(img[:, :, 2]).reshape(-1, 1)
        # print(img.shape, r.shape)
        st = ''
        for i in range(r.shape[0]):
            st = st + hex(int(r[i][0]+g[i][0]+b[i][0], 2))[2:] + ',\n'
    elif RGBMode == 1:
        grayImg = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        # plt.imshow(grayImg, cmap='gray')
        # plt.show()
        grayImg = grayImg.reshape(-1, 1)

        st = ''
        for i in range(grayImg.shape[0]):
            st = st + hex(grayImg[i][0])[2:] + ',\n'

    elif RGBMode == 2:
        print('rgbmode 2 ')
        r = img[:, :, 0].reshape(-1, 1)
        g = img[:, :, 1].reshape(-1, 1)
        b = img[:, :, 2].reshape(-1, 1)
        st = ''
        for i in range(r.shape[0]):
            st = st + hex(r[i][0])[2:].zfill(2)+hex(g[i][0]
                                                    )[2:].zfill(2)+hex(b[i][0])[2:].zfill(2) + ',\n'

    with open('./ImageMemoryFile/MemoryFile.coe', 'w+') as f:
        f.write('memory_initialization_radix=16;\nmemory_initialization_vector=\n')
        f.write(st[:-2]+';')
        f.close()

In [4]:

def MemoryFileToImg(fileName, WH={'W': 240, 'H': 120}, RGBMode=0):
    lisa = None
    with open(fileName) as fil:
        lisa = list(map(lambda x: x.strip()[:-1], fil.readlines()))
        fil.close()
    lisa = np.asarray(lisa[2:])

    if RGBMode == 0:
        dummyImg = np.zeros((WH['H'], WH['W'], 3))

        for j in range(WH['H']):
            for i in range(WH['W']):
                addr = (j*WH['W']) + i
                ghost = SixteenbitRGBtoSeperateRGB(lisa[addr])
                dummyImg[j, i] = ghost
        dummyImg = np.asarray(dummyImg, dtype=np.uint8)

        plt.imshow(dummyImg)
        plt.show()
    elif RGBMode == 1:
        dummyImg = np.zeros((WH['H'], WH['W']))

        for j in range(WH['H']):
            for i in range(WH['W']):
                addr = (j*WH['W']) + i
                ghost = lisa[addr]
                dummyImg[j, i] = int('0x'+ghost, 16)
        dummyImg = np.asarray(dummyImg, dtype=np.uint8)

        plt.imshow(dummyImg, cmap='gray')
        plt.show()

    elif RGBMode == 2:
        dummyImg = np.zeros((WH['H'], WH['W'], 3))

        for j in range(WH['H']):
            for i in range(WH['W']):
                addr = (j*WH['W']) + i
                ghost = np.asarray(
                    [int('0x'+lisa[addr][0:2], 16), int('0x'+lisa[addr][2:4], 16), int('0x'+lisa[addr][4:6], 16)])
                # print(ghost)
                dummyImg[j, i] = ghost
        dummyImg = np.asarray(dummyImg, dtype=np.uint8)

        plt.imshow(dummyImg)
        plt.show()

In [13]:
RGBMode = 1
WH = {'W': 320, 'H': 240}
imgToMemoryFile('cartoonImg.jpg', WH, RGBMode=RGBMode)
MemoryFileToImg('./ImageMemoryFile/MemoryFile.coe',
                    WH, RGBMode=RGBMode)

In [6]:
def byteIntTobyte(intVal):
    # stringVal = str(intVal)
    byteVal=intVal.to_bytes(1,'little')
    return byteVal
# print(byteIntTobyte(57))
def IntTobyteArray(intVal):
    firstByte = intVal & 0xFF
    secondByte = (intVal>>8) & 0xFF
    thirdByte = (intVal>>16) & 0xFF
    fourthByte = (intVal>>24) & 0xFF
    retAr = []
    retAr.append(byteIntTobyte(firstByte))
    retAr.append(byteIntTobyte(secondByte))
    retAr.append(byteIntTobyte(thirdByte))
    retAr.append(byteIntTobyte(fourthByte))
    return retAr

In [7]:
def sendDepth(ser, depth=0, delay=0.01):
    dataBytest = IntTobyteArray(depth)
    # print(dataBytest, dataBytest[0])
    ser.write(b'\x0F')
    time.sleep(delay)
    ser.write(dataBytest[0])
    time.sleep(delay)
    ser.write(dataBytest[1])
    time.sleep(delay)
    ser.write(dataBytest[2])
    time.sleep(delay)
    ser.write(dataBytest[3])
    time.sleep(delay)

def getDepth(ser, delay=0.001):
    ser.write(b'\x0E')
    a = ser.read(4)
    int_val = int.from_bytes(a, "little")
    return int_val

In [8]:
def sendDataByte(ser, byteVal=0, delay=0.01):
    gg = byteIntTobyte(byteVal)
    ser.write(b'\x0C')
    time.sleep(delay)
    ser.write(gg)
    time.sleep(delay)    
def sendBRAMData(ser, delay=0.01, fileName = ''):
    lisa = None
    with open(fileName) as fil:
        lisa = list(map(lambda x: x.strip()[:-1], fil.readlines()))
        fil.close()
    lisa = lisa[2:]
    
    sendDepth(ser, depth=len(lisa), delay=delay)
    print(getDepth(ser))

    ser.write(b'\x0C')
    time.sleep(delay)
    # print(lisa)
    for i in range(len(lisa)):
        val = int('0x'+lisa[i],16)
        # print(val)
        ser.write(byteIntTobyte(val))
        time.sleep(delay)     

In [12]:
ser = serial.Serial('/dev/ttyUSB3', 115200)
sendBRAMData(ser, fileName='./ImageMemoryFile/MemoryFile.coe', delay=0.0002)
ser.close()

76800
